<a href="https://colab.research.google.com/github/ashivashankars/CMPE256_Assignments/blob/main/Jobs_with_Feature_extraction(job_type)_%26_Chatbot%20final%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai
!pip install -q PyPDF2
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 96.5 MB/s eta 0:00:00


In [ ]:
import os
import json
import time
import sys
import re
import pandas as pd
import requests
import faiss
import numpy as np
import gradio

from openai import OpenAI
import PyPDF2
from google.colab import userdata
from google.colab import files
from google.colab import drive
from bs4 import BeautifulSoup


import gradio as gr
import json
from openai import OpenAI
from google.colab import userdata

# Install pypdf if not already installed
try:
    from pypdf import PdfReader
except ImportError:
    !pip install pypdf
    from pypdf import PdfReader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Define Google Drive path for caching
DRIVE_PATH = '/content/drive/MyDrive/job_matching_cache'

# --- 1. SECURE API KEY HANDLING ---
try:
    # Tries to get key from Colab Secrets (Left sidebar > Key icon)
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not found in Colab Secrets.")
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI API Key loaded securely from Colab Secrets.")
except Exception as e:
    print(f"Error: {e}")
    print("Please click the Key icon on the left, add 'OPENAI_API_KEY', and enable notebook access.")
    sys.exit(1)

# --- 2. CONFIGURATION & HELPER FUNCTIONS ---
# (Manual file upload removed as it is handled by the Gradio App)

FIELD_CONFIG = {
    "graduation_date":      {"id": 1, "text": "Expected Graduation Date", "mode": "interactive"},
    "current_degree_major": {"id": 2, "text": "Current Degree AND Major (e.g. MS in CS)", "mode": "interactive"},
    "current_degree_gpa":   {"id": 3, "text": "GPA (For Current Degree Only)", "mode": "interactive"},
    "us_citizenship":       {"id": 4, "text": "Are you a US Citizen? (If No, specify Visa Type & Sponsorship)", "mode": "interactive"},
    "programming_languages": {"id": 5, "text": "Programming Languages", "mode": "extract"},
    "experience_software":   {"id": 6, "text": "Work/Project Experience Summary", "mode": "extract"},
    "tools_frameworks":      {"id": 7, "text": "Tools & Frameworks", "mode": "extract"},
    "leadership":            {"id": 8, "text": "Leadership Experience", "mode": "extract"},
    "job_preference":        {"id": 9, "text": "Looking for Full-time / Internship / Both?", "mode": "interactive"},
    "impact_outcomes":       {"id": 10,"text": "Quantifiable Impact & Key Achievements", "mode": "extract"}
}

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None
    return text

def save_embeddings_and_index(index, prepared_job_data_to_save, drive_path):
    print("\n--- Saving embeddings and FAISS index to Google Drive ---")
    drive.mount('/content/drive', force_remount=True)

    os.makedirs(drive_path, exist_ok=True)

    faiss_index_path = os.path.join(drive_path, 'faiss_index.bin')
    job_data_path = os.path.join(drive_path, 'prepared_job_data.json')

    try:
        faiss.write_index(index, faiss_index_path)
        print(f"FAISS index saved to {faiss_index_path}")

        # Prepare prepared_job_data for JSON serialization
        # Convert numpy arrays (job_embedding) to lists
        serializable_job_data = []
        for job_dict in prepared_job_data_to_save:
            temp_job_dict = job_dict.copy()
            if 'job_embedding' in temp_job_dict and isinstance(temp_job_dict['job_embedding'], np.ndarray):
                temp_job_dict['job_embedding'] = temp_job_dict['job_embedding'].tolist()
            serializable_job_data.append(temp_job_dict)

        with open(job_data_path, 'w') as f:
            json.dump(serializable_job_data, f, indent=4)
        print(f"Prepared job data saved to {job_data_path}")
    except Exception as e:
        print(f"Error saving cache files: {e}")

def load_embeddings_and_index(drive_path):
    print("\n--- Attempting to load embeddings and FAISS index from Google Drive ---")
    drive.mount('/content/drive', force_remount=True)

    faiss_index_path = os.path.join(drive_path, 'faiss_index.bin')
    job_data_path = os.path.join(drive_path, 'prepared_job_data.json')

    loaded_index = None
    loaded_prepared_job_data = None

    try:
        if os.path.exists(faiss_index_path) and os.path.exists(job_data_path):
            loaded_index = faiss.read_index(faiss_index_path)
            print(f"FAISS index loaded from {faiss_index_path}")

            with open(job_data_path, 'r') as f:
                loaded_prepared_job_data = json.load(f)
            print(f"Prepared job data loaded from {job_data_path}")

            # Convert job_embedding back to numpy array
            for job_dict in loaded_prepared_job_data:
                if 'job_embedding' in job_dict and isinstance(job_dict['job_embedding'], list):
                    job_dict['job_embedding'] = np.array(job_dict['job_embedding'], dtype='float32')

            print("Cache loaded successfully.")
            return loaded_index, loaded_prepared_job_data
        else:
            raise FileNotFoundError("Cache files not found.")
    except FileNotFoundError:
        print("Cache files (FAISS index or prepared job data) not found in Google Drive. Will generate new ones.")
        return None, None
    except Exception as e:
        print(f"Error loading cache files: {e}")
        return None, None

OpenAI API Key loaded securely from Colab Secrets.


In [ ]:
print("------------------------------")
print("--- Pre-loading Job Data and Initializing FAISS Index (with Caching) ---")

# Attempt to load from cache first
try:
    loaded_index, loaded_prepared_job_data = load_embeddings_and_index(DRIVE_PATH)
except Exception as e:
    print(f"Error loading from Google Drive cache: {e}. Proceeding without cache.")
    loaded_index, loaded_prepared_job_data = None, None

if loaded_index is not None and loaded_prepared_job_data is not None:
    index = loaded_index
    prepared_job_data = loaded_prepared_job_data
    # Ensure job_type is added if loading from an older cache that didn't save it
    for j in prepared_job_data:
        if 'job_type' not in j:
            r = j.get('role', '')
            if re.search(r'intern', r, re.IGNORECASE): j['job_type'] = 'Intern'
            elif re.search(r'new grad|graduate', r, re.IGNORECASE): j['job_type'] = 'New Grad'
            else: j['job_type'] = 'Full-time'
    print("Data and FAISS index successfully loaded from Google Drive cache.")
else:
    print("Cache not found or failed to load. Proceeding with fresh data generation.")
    # --- 1. Fetch Job Postings and Parse ---
    # Define the GitHub raw URL
    github_raw_url = "https://raw.githubusercontent.com/SimplifyJobs/Summer2026-Internships/dev/README.md"

    # Fetch the content of the README.md file
    try:
        response = requests.get(github_raw_url)
        response.raise_for_status() # Raise an exception for HTTP errors
        readme_content = response.text
        print(f"Successfully fetched README content from {github_raw_url}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching README content: {e}")
        readme_content = "" # Initialize with empty string on error

    # Define a regular expression pattern to find level 2 headings
    heading_pattern = re.compile(r"^##\s(.+)$", re.MULTILINE)

    # Find all matching headings in the readme_content
    section_headings = heading_pattern.findall(readme_content)

    # Initialize an empty dictionary to store extracted DataFrames
    all_sections_data = {}

    # Iterate through each section heading to extract its content
    for i, current_heading in enumerate(section_headings):
        current_heading_full = f"## {current_heading}"
        start_index = readme_content.find(current_heading_full)

        if start_index == -1:
            continue

        end_index = -1
        if i + 1 < len(section_headings):
            next_heading_full = f"## {section_headings[i+1]}"
            end_index = readme_content.find(next_heading_full, start_index + len(current_heading_full))

        if end_index != -1:
            section_content = readme_content[start_index + len(current_heading_full):end_index].strip()
        else:
            section_content = readme_content[start_index + len(current_heading_full):].strip()

        section_dfs = [] # To store multiple tables if a section has them

        soup = BeautifulSoup(section_content, 'lxml')
        tables = soup.find_all('table')

        for table in tables:
            headers = []
            if table.find('thead'):
                for th in table.find('thead').find_all('th'):
                    headers.append(th.get_text(strip=True))

            data_rows = []
            if table.find('tbody'):
                for tr in table.find('tbody').find_all('tr'):
                    row_values = []
                    for idx, td in enumerate(tr.find_all('td')):
                        if headers and idx < len(headers) and headers[idx] == 'Application':
                            link = td.find('a')
                            if link and 'href' in link.attrs:
                                row_values.append(link['href'])
                            else:
                                row_values.append('')
                        else:
                            row_values.append(td.get_text(strip=True))

                    if len(headers) > 0:
                        if len(row_values) > len(headers):
                            row_values = row_values[:len(headers)]
                        elif len(row_values) < len(headers):
                            row_values.extend([''] * (len(headers) - len(row_values))) # Pad if too few
                        data_rows.append(row_values)

            if headers and data_rows:
                df = pd.DataFrame(data_rows, columns=headers)
                section_dfs.append(df)

        if section_dfs:
            all_sections_data[current_heading] = section_dfs[0]
        else:
            all_sections_data[current_heading] = None
    print(f"Extracted data for {len(all_sections_data)} sections from GitHub README.")

    # --- 2. Prepare Job Data for Embedding ---
    prepared_job_data = []

    for section_name, df in all_sections_data.items():
        if df is not None:
            df_normalized = df.copy()
            df_normalized.columns = df_normalized.columns.str.lower()

            # FIXED: Renamed loop variable from 'index' to 'idx' to avoid shadowing the global FAISS index variable
            for idx, row in df_normalized.iterrows():
                company = row['company'] if 'company' in row and pd.notna(row['company']) and str(row['company']).strip() != '' else 'N/A'
                role = row['role'] if 'role' in row and pd.notna(row['role']) and str(row['role']).strip() != '' else 'N/A'
                location = row['location'] if 'location' in row and pd.notna(row['location']) and str(row['location']).strip() != '' else 'N/A'
                application_link = row['application'] if 'application' in row and pd.notna(row['application']) and str(row['application']).strip() != '' else 'N/A'

                embedding_text_parts = [
                    f"Company: {company}",
                    f"Role: {role}",
                    f"Location: {location}",
                    f"Section: {section_name}"
                ]
                embedding_text = ". ".join(embedding_text_parts)

                job_details = row.to_dict()
                job_details['application_link'] = application_link

                if 'application' in job_details:
                    del job_details['application']

                job_details['section'] = section_name
                job_details['description'] = 'N/A'
                job_details['embedding_text'] = embedding_text

                # Assign job_type based on role, to be saved with the job_details
                r = job_details.get('role', '')
                if re.search(r'intern', r, re.IGNORECASE): job_details['job_type'] = 'Intern'
                elif re.search(r'new grad|graduate', r, re.IGNORECASE): job_details['job_type'] = 'New Grad'
                else: job_details['job_type'] = 'Full-time'

                prepared_job_data.append(job_details)
    print(f"Prepared {len(prepared_job_data)} job entries for embedding.")

    # --- 3. Generate Embeddings for Job Postings ---
    EMBEDDING_MODEL = "text-embedding-ada-002" # Define the embedding model
    job_embeddings = []

    for i, job in enumerate(prepared_job_data):
        try:
            job_embedding_response = client.embeddings.create(
                model=EMBEDDING_MODEL,
                input=job['embedding_text']
            )
            job['job_embedding'] = job_embedding_response.data[0].embedding
            job_embeddings.append(job['job_embedding'])
            if i % 100 == 0: # Print progress every 100 jobs
                print(f"Generated embedding for job {i+1}/{len(prepared_job_data)}")
        except Exception as e:
            print(f"Error generating embedding for job {i+1}: {e}")
            job['job_embedding'] = None
    print(f"Generated embeddings for {len(job_embeddings)} job postings.")

    # --- 4. Initialize FAISS Index and Add Job Embeddings ---
    embedding_dimension = 1536 # OpenAI's 'text-embedding-ada-002' dimension is 1536
    index = faiss.IndexFlatL2(embedding_dimension)

    valid_job_embeddings = [job['job_embedding'] for job in prepared_job_data if 'job_embedding' in job and job['job_embedding'] is not None]
    job_embeddings_np = np.array(valid_job_embeddings).astype('float32')

    if len(job_embeddings_np) > 0:
        index.add(job_embeddings_np)
        print(f"FAISS index initialized and {index.ntotal} job embeddings added.")
    else:
        print("No valid job embeddings to add to the FAISS index.")

    # Save the newly generated data to cache
    save_embeddings_and_index(index, prepared_job_data, DRIVE_PATH)

print("Pre-loading complete.")
print("------------------------------")

# Verification steps:
print(f"Number of job entries in prepared_job_data: {len(prepared_job_data)}")
print(f"Total vectors in FAISS index: {index.ntotal}")

------------------------------
--- Pre-loading Job Data and Initializing FAISS Index (with Caching) ---

--- Attempting to load embeddings and FAISS index from Google Drive ---
Mounted at /content/drive
FAISS index loaded from /content/drive/MyDrive/job_matching_cache/faiss_index.bin
Prepared job data loaded from /content/drive/MyDrive/job_matching_cache/prepared_job_data.json
Cache loaded successfully.
Data and FAISS index successfully loaded from Google Drive cache.
Pre-loading complete.
------------------------------
Number of job entries in prepared_job_data: 1140
Total vectors in FAISS index: 1140


# Task
To debug and enhance the job matching process, I will modify the `perform_job_search` function to include detailed logging of candidate data, matching results, and output string construction. Additionally, I will improve the no-match feedback message to provide more specific reasons for zero results. This will enable verification of Gradio UI output and provide insights into the matching logic.

## Add Debugging Logs for Matching Criteria

### Subtask:
Insert detailed logging in `perform_job_search` to show the `current_data` (especially 'job_preference' and 'graduation_date') and `current_profile_text` being used for matching. Crucially, log the `matches` and `threshold` variables immediately after calling `find_matches_with_criteria` to confirm if any jobs are being identified at that stage.


**Reasoning**:
To add debugging logs as requested, I need to modify the `perform_job_search` function in the existing code cell `dd93c5b9` by inserting print statements at the specified locations. This will help in understanding the input parameters for job matching and the immediate results after the matching process.



In [ ]:
import json
import re
from PyPDF2 import PdfReader
import gradio as gr
import numpy as np
from openai import OpenAI

# 1. SETUP & CONFIGURATION
# ---------------------------------------------------------
try:
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")  # userdata may be provided by your environment
    client = OpenAI(api_key=OPENAI_API_KEY)
except Exception as e:
    print(f"Warning: API Key not found. {e}")
    client = None

# Defines the schema we want to extract
TARGET_SCHEMA = {
    "graduation_date": "NULL",
    "current_degree_major": "NULL",
    "current_degree_gpa": "NULL",  # New field
    "us_citizenship": "NULL",
    "visa_type": "NULL",  # Re-added field for visa type
    "programming_languages": "NULL",  # New field, replaces technical_skills
    "experience_software": "NULL",  # New field
    "tools_frameworks": "NULL",  # New field
    "leadership": "NULL",  # New field
    "job_preference": "NULL"
}

# 1b. SANITIZATION HELPERS
# ---------------------------------------------------------
def sanitize_string(text):
    """
    Sanitizes a string by converting it to UTF-8 and back, replacing invalid characters.
    This helps prevent TypeErrors in Gradio due to non-UTF-8 compatible strings.
    """
    if text is None:
        return ""
    s = str(text)
    encoded_bytes = s.encode("utf-8", errors="replace")
    sanitized_s = encoded_bytes.decode("utf-8")
    return sanitized_s


def sanitize_structure(obj):
    """
    Recursively sanitize any strings in nested structures (dicts, lists, tuples)
    to ensure they are valid UTF-8.
    """
    if isinstance(obj, str):
        return sanitize_string(obj)
    elif isinstance(obj, dict):
        return {k: sanitize_structure(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sanitize_structure(v) for v in obj]
    elif isinstance(obj, tuple):
        return tuple(sanitize_structure(v) for v in obj)
    else:
        return obj


# 2. AGENT 1: THE EXTRACTOR
# ---------------------------------------------------------
def agent_extractor(file_path):
    """
    Reads PDF and uses LLM to extract initial JSON data.
    """
    # Step A: OCR / Text Extraction
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    except Exception as e:
        return {"error": f"PDF Read Error: {e}"}

    # Step B: LLM Extraction
    prompt = f"""
    You are a Resume Parser Agent. Extract the following fields from the resume text.
    Return ONLY valid JSON matching this structure exactly.
    Use the string "NULL" if the information is not explicitly found.
    For 'leadership', if found, output a list of dictionaries with 'role', 'organization', and 'description'. Otherwise, output 'NULL'.

    Target Structure:
    {json.dumps(TARGET_SCHEMA, indent=2)}

    RESUME TEXT:
    {text[:4000]}
    """

    if client is None:
        print("Extraction Error: OpenAI client is not initialized.")
        return TARGET_SCHEMA

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a JSON extractor."},
                {"role": "user", "content": prompt},
            ],
            response_format={"type": "json_object"},
        )
        extracted_data = json.loads(response.choices[0].message.content)
        extracted_data = sanitize_structure(extracted_data)
        return extracted_data
    except Exception as e:
        print(f"Extraction Error: {e}")
        return TARGET_SCHEMA  # Return empty schema on fail to prevent crash


# 3. AGENT 2: THE INTERVIEWER
# ---------------------------------------------------------
def agent_interviewer(current_data, user_response=None, current_field=None):
    """
    Analyzes data, updates with user response, and determines the next question.
    Returns: (updated_data, next_field_to_ask, question_text)
    """

    # 1. Update data if user provided an answer
    if user_response and current_field:
        current_data[current_field] = user_response

    # Special handling for us_citizenship = "no"
    if current_field == "us_citizenship" and user_response and user_response.lower() in [
        "no",
        "n",
        "false",
    ]:
        if current_data.get("visa_type") in ["NULL", "", None, "null"]:
            current_data["visa_type"] = "NULL"  # Explicitly mark as missing to ensure it's asked next
    # If user says yes to citizenship, ensure visa_type is not asked
    elif current_field == "us_citizenship" and user_response and user_response.lower() in [
        "yes",
        "y",
        "true",
    ]:
        current_data["visa_type"] = "N/A"  # Not applicable if US Citizen

    # 2. Find the next missing field
    next_field = None
    question = None

    fields_to_check = list(TARGET_SCHEMA.keys())

    for field in fields_to_check:
        val = current_data.get(field)
        if val in ["NULL", "", None, "null"]:
            next_field = field

            human_field = field.replace("_", " ").title()
            question = (
                f"I noticed your resume is missing **{human_field}**. could you please provide that?"
            )

            if field == "us_citizenship":
                question = "Are you a **US Citizen**? (Yes/No)"
            elif field == "graduation_date":
                question = "When is your expected **Graduation Date**?"
            elif field == "current_degree_gpa":
                question = "What is your **GPA** for your Current Degree Only?"
            elif field == "job_preference":
                question = "What is your specific **Job Preference** (e.g., Full-time / Internship / Both)?"
            elif field == "visa_type":
                question = "What is your **Visa Type** (e.g., H1B, F1-OPT)? Or if you don't need sponsorship, please state that."

            break

    current_data = sanitize_structure(current_data)
    if question is not None:
        question = sanitize_string(question)

    return current_data, next_field, question


# 4. AGENT 3: THE FINALIZER
# ---------------------------------------------------------
def agent_finalizer(final_data):
    """
    Clean up data and generate a file for download.
    """
    final_data = sanitize_structure(final_data)

    filename = "candidate_profile.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(final_data, f, indent=4, ensure_ascii=False)

    msg = (
        "**Interview Complete!**\n\nI have generated your profile.\n\n```json\n"
        + json.dumps(final_data, indent=2, ensure_ascii=False)
        + "\n```"
    )
    msg = sanitize_string(msg)
    return filename, msg


# Helper function to generate profile text for embedding
def _generate_profile_text(data):
    data = sanitize_structure(data)
    profile_parts = []
    for key in TARGET_SCHEMA.keys():
        val = data.get(key)
        if val not in ["NULL", "", None, "null", "N/A"]:
            if isinstance(val, list) and key == "leadership":
                roles = [item.get("role", "") for item in val if item.get("role")]
                if roles:
                    profile_parts.append(
                        f"{key.replace('_', ' ').title()}: {'; '.join(roles)}"
                    )
            else:
                profile_parts.append(
                    f"{key.replace('_', ' ').title()}: {sanitize_string(val)}"
                )
    return ". ".join(profile_parts)


# Job Matching Helper Functions
# ---------------------------------------------------------
def hybrid_search_jobs(resume_embedding, candidate_preferences, all_jobs):
    pref = str(candidate_preferences.get("job_preference", "")).lower()
    grad_date_str = str(candidate_preferences.get("graduation_date", "")).lower()

    allowed_types = set()
    if "internship" == pref:
        allowed_types.add("Intern")
    elif "full-time" == pref:
        allowed_types.update(["Full-time", "New Grad"])
    elif "both" == pref:
        allowed_types.update(["Intern", "Full-time", "New Grad"])
    else:
        import datetime
        curr_year = datetime.datetime.now().year
        years = [int(y) for y in re.findall(r"\b\d{4}\b", grad_date_str)]
        if years and max(years) > curr_year:
            allowed_types = {"Intern", "New Grad"}
        else:
            allowed_types = {"Full-time", "New Grad"}

    valid_jobs = [
        j
        for j in all_jobs
        if j.get("job_embedding") is not None
        and j.get("job_type", "Full-time") in allowed_types
    ]
    print(f"[DEBUG] hybrid_search_jobs: valid_jobs after filtering = {len(valid_jobs)}")
    if not valid_jobs:
        return []

    resume_vec = np.array(resume_embedding, dtype="float32").flatten()
    job_matrix = np.array([j["job_embedding"] for j in valid_jobs], dtype="float32")

    sims = np.dot(job_matrix, resume_vec) / (
        np.linalg.norm(resume_vec) * np.linalg.norm(job_matrix, axis=1) + 1e-10
    )

    results = [
        {"job_details": j, "cosine_similarity": float(s)}
        for j, s in zip(valid_jobs, sims)
    ]
    results.sort(key=lambda x: x["cosine_similarity"], reverse=True)
    print(f"[DEBUG] hybrid_search_jobs: total results (sorted) = {len(results)}")
    return results


def find_matches_with_criteria(verified_prefs, profile_text, client_openai, prepared_jobs):
    try:
        emb = (
            client_openai.embeddings.create(
                model="text-embedding-ada-002", input=profile_text
            )
            .data[0]
            .embedding
        )
    except Exception as e:
        print(f"Embedding generation failed: {e}")
        return [], 0.0

    ranked = hybrid_search_jobs(emb, verified_prefs, prepared_jobs)
    print(f"[DEBUG] find_matches_with_criteria: ranked jobs count = {len(ranked)}")

    if not ranked:
        return [], 0.0

    # Option 3: keep full ranked list; pagination will handle display
    top_matches = ranked
    threshold = top_matches[-1]["cosine_similarity"]

    print(f"[DEBUG] find_matches_with_criteria: top_matches count (full list) = {len(top_matches)}")
    return top_matches, threshold


def augment_job_with_llm(job_match, profile_text, client_openai):
    job = job_match["job_details"]
    if client_openai is None:
        job_match["llm_fit_summary"] = "Analysis not available (OpenAI client missing)."
        return job_match

    try:
        res = client_openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": (
                        "Explain why candidate fits job in 2 sentences.\n"
                        f"Candidate: {profile_text}\nJob: {job.get('embedding_text','')}"
                    ),
                }
            ],
            max_tokens=100,
        )
        job_match["llm_fit_summary"] = sanitize_string(
            res.choices[0].message.content.strip()
        )
    except Exception as e:
        print(f"LLM augmentation failed: {e}")
        job_match["llm_fit_summary"] = "Analysis failed."
    return job_match


# 5. GRADIO ORCHESTRATOR (UI LOGIC)
# ---------------------------------------------------------
def process_upload(file, history, state):
    """Triggered when file is uploaded"""
    if not file:
        return history, state, None, ""

    extracted_data = agent_extractor(file)

    initial_profile_text = _generate_profile_text(extracted_data)
    initial_profile_text = sanitize_string(initial_profile_text)

    updated_data, next_field, question = agent_interviewer(extracted_data)

    # Reset job-related state on new upload
    state["job_md_list"] = []
    state["job_page"] = 0

    state["data"] = sanitize_structure(updated_data)
    state["current_field"] = next_field
    state["profile_text"] = initial_profile_text

    history.append((None, sanitize_string("Resume parsed! Checking for missing info...")))
    if question:
        history.append((None, sanitize_string(question)))
        return history, state, None, ""
    else:
        fname, msg = agent_finalizer(updated_data)
        history.append((None, sanitize_string(msg)))
        return history, state, fname, ""


def process_chat(user_msg, history, state):
    """Triggered when user types a message"""
    if not user_msg:
        return history, state, None

    user_msg = sanitize_string(user_msg)
    history.append((user_msg, None))

    data = state.get("data", TARGET_SCHEMA.copy())
    current_field = state.get("current_field")

    updated_data, next_field, question = agent_interviewer(
        data, user_msg, current_field
    )

    updated_profile_text = _generate_profile_text(updated_data)
    updated_profile_text = sanitize_string(updated_profile_text)

    state["data"] = sanitize_structure(updated_data)
    state["current_field"] = next_field
    state["profile_text"] = updated_profile_text

    # Reset job pagination on profile change
    state["job_md_list"] = []
    state["job_page"] = 0

    if next_field:
        history.append((None, sanitize_string(question)))
        return history, state, None
    else:
        filename, final_msg = agent_finalizer(updated_data)
        history.append((None, sanitize_string(final_msg)))
        return history, state, filename


PAGE_SIZE = 5  # number of jobs per page


def perform_job_search(state_obj, use_llm_fit, progress=gr.Progress()):
    """
    Performs the job search based on the current state data.
    If use_llm_fit is True, only the FIRST PAGE (first 5 jobs) will call LLM for fit explanation.
    """
    global client, prepared_job_data, index, DRIVE_PATH

    current_data = state_obj.get("data", TARGET_SCHEMA.copy())
    current_profile_text = state_obj.get("profile_text", "")

    state_obj["job_md_list"] = []
    state_obj["job_page"] = 0

    hide_load_more = gr.update(visible=False)

    if (
        not current_profile_text
        or not current_data.get("graduation_date")
        or client is None
    ):
        msg = (
            "Please upload a resume, complete the interview, "
            "and ensure OpenAI client is initialized."
        )
        print("[DEBUG] perform_job_search: missing profile_text or graduation_date or client.")
        return msg, state_obj, hide_load_more

    # Load job data + index if needed
    if (
        "prepared_job_data" not in globals()
        or prepared_job_data is None
        or "index" not in globals()
        or index is None
    ):
        print("[DEBUG] perform_job_search: job data or index not found, loading from cache...")
        try:
            # Assumes load_embeddings_and_index and DRIVE_PATH are defined externally
            new_index, new_prepared_job_data = load_embeddings_and_index(DRIVE_PATH)
            if new_index is not None and new_prepared_job_data is not None:
                index = new_index
                prepared_job_data = new_prepared_job_data
                for j in prepared_job_data:
                    if "job_type" not in j:
                        r = j.get("role", "")
                        if re.search(r"intern", r, re.IGNORECASE):
                            j["job_type"] = "Intern"
                        elif re.search(r"new grad|graduate", r, re.IGNORECASE):
                            j["job_type"] = "New Grad"
                        else:
                            j["job_type"] = "Full-time"
                print("[DEBUG] perform_job_search: job data and index successfully loaded from cache.")
            else:
                print("[DEBUG] perform_job_search: failed to load job data or index from cache.")
                return (
                    "Error: Job data and FAISS index could not be loaded from cache. "
                    "Please re-run pre-loading steps.",
                    state_obj,
                    hide_load_more,
                )
        except Exception as e:
            print(f"[DEBUG] perform_job_search: exception while loading job data: {e}")
            return (
                f"Error loading job data and index from cache: {e}. "
                "Please re-run pre-loading steps.",
                state_obj,
                hide_load_more,
            )

    if client is None or prepared_job_data is None or index is None:
        print("[DEBUG] perform_job_search: client or prepared_job_data or index still None after load.")
        return (
            "Error: OpenAI client or job data not initialized even after attempted load. "
            "Please ensure previous cells run successfully.",
            state_obj,
            hide_load_more,
        )

    matches, threshold = find_matches_with_criteria(
        current_data, current_profile_text, client, prepared_job_data
    )

    print(f"[DEBUG] perform_job_search: matches count = {len(matches)}, threshold = {threshold:.4f}")
    print(f"[DEBUG] perform_job_search: use_llm_fit = {use_llm_fit}")

    if not matches:
        feedback_message = (
            "No job matches found based on your profile. This might be due to:\n"
            "- No suitable job types (Intern, Full-time, New Grad) matching your preference.\n"
            "- Lack of strong skill alignment in your profile.\n"
            "- Your graduation date not aligning with available opportunities."
        )
        return feedback_message, state_obj, hide_load_more

    job_md_list = []
    total_matches = len(matches)

    for i, m in enumerate(matches):
        progress((i + 1) / total_matches, desc=f"Preparing job {i+1}/{total_matches}...")

        # OPTION 3: Only call LLM for first PAGE_SIZE jobs if requested
        fit_summary = "High similarity based on your profile."
        if use_llm_fit and i < PAGE_SIZE:
            m = augment_job_with_llm(m, current_profile_text, client)
            fit_summary = m.get("llm_fit_summary", fit_summary)

        job = m["job_details"]

        chunk = ""
        chunk += (
            f"### {i+1}. {sanitize_string(job.get('company','N/A'))} - "
            f"{sanitize_string(job.get('role','N/A'))}\n"
        )
        chunk += (
            f"**Loc:** {sanitize_string(job.get('location','N/A'))} | "
            f"**Sim:** {m['cosine_similarity']*100:.1f}%\n"
        )
        chunk += f"**Fit:** {sanitize_string(fit_summary)}\n"
        chunk += f"[Apply]({sanitize_string(job.get('application_link','#'))})\n\n---\n\n"

        job_md_list.append(chunk)

    print(f"[DEBUG] perform_job_search: job_md_list length (jobs to display) = {len(job_md_list)}")

    state_obj["job_md_list"] = job_md_list
    state_obj["job_page"] = 1  # first page

    total = len(job_md_list)
    start = 0
    end = min(PAGE_SIZE, total)
    first_jobs_md = "".join(job_md_list[start:end])
    header = f"### 🎯 Found {total} matching jobs (showing {end} of {total}):\n---\n\n"
    output_str = header + first_jobs_md

    more_available = end < total
    load_more_update = gr.update(visible=more_available)
    print(f"[DEBUG] perform_job_search: Load More visible = {more_available}")

    return output_str, state_obj, load_more_update


def load_more_jobs(current_output, state_obj):
    """
    Appends the next page of jobs to the existing Markdown output.
    Returns: (updated_output, updated_state, load_more_button_update)
    """
    job_md_list = state_obj.get("job_md_list", [])
    page = state_obj.get("job_page", 0)
    total = len(job_md_list)

    print(f"[DEBUG] load_more_jobs: current page = {page}, total jobs = {total}")

    if not job_md_list or page <= 0:
        print("[DEBUG] load_more_jobs: no jobs to paginate or invalid page.")
        return current_output, state_obj, gr.update(visible=False)

    start = page * PAGE_SIZE
    end = min(start + PAGE_SIZE, total)

    if start >= total:
        print("[DEBUG] load_more_jobs: start index >= total, no more jobs.")
        return current_output, state_obj, gr.update(visible=False)

    new_chunk = "".join(job_md_list[start:end])
    updated_output = current_output + new_chunk

    state_obj["job_page"] = page + 1

    more_available = end < total
    load_more_update = gr.update(visible=more_available)

    print(f"[DEBUG] load_more_jobs: showing jobs {start+1}-{end} of {total}, Load More visible = {more_available}")

    return updated_output, state_obj, load_more_update


# 6. UI CONSTRUCTION
# ---------------------------------------------------------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    state = gr.State({
        "data": {},
        "current_field": None,
        "profile_text": "",
        "job_md_list": [],
        "job_page": 0
    })

    gr.Markdown("# 🤖 Agentic Resume Screener")
    gr.Markdown("Upload a resume. The agents will extract data and interview you for missing details.")

    with gr.Row():
        with gr.Column(scale=1):
            file_upload = gr.File(label="1. Upload Resume (PDF)", type="filepath")
            download_btn = gr.File(label="3. Download Profile", interactive=False)

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                height=300, label="2. Interview Agent", bubble_full_width=False
            )
            msg_input = gr.Textbox(
                label="Your Answer", placeholder="Type here and press enter..."
            )

    with gr.Row():
        use_llm_fit_checkbox = gr.Checkbox(
            label="Explain fit with AI (slower for first page)", value=True
        )
        find_jobs_btn = gr.Button("4. Find Jobs", variant="primary")
        load_more_btn = gr.Button("Load More Jobs", visible=False)

    job_output_display = gr.Markdown("", label="Matching Jobs")

    # Event: File Upload
    file_upload.change(
        fn=process_upload,
        inputs=[file_upload, chatbot, state],
        outputs=[chatbot, state, download_btn, job_output_display],
    )

    # Event: User Chat
    msg_input.submit(
        fn=process_chat,
        inputs=[msg_input, chatbot, state],
        outputs=[chatbot, state, download_btn],
    ).then(lambda: "", outputs=msg_input)

    # Event: Find Jobs (initial search + first page)
    find_jobs_btn.click(
        fn=perform_job_search,
        inputs=[state, use_llm_fit_checkbox],
        outputs=[job_output_display, state, load_more_btn],
    )

    # Event: Load More Jobs (pagination)
    load_more_btn.click(
        fn=load_more_jobs,
        inputs=[job_output_display, state],
        outputs=[job_output_display, state, load_more_btn],
    )

demo.launch(debug=True)


/tmp/ipython-input-3322284446.py:569: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3322284446.py:587: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3322284446.py:587: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Gradio 6.0. This parameter no longer has any effect.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3322284446.py:587: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=F

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4a7e95a7af8e5af58d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[DEBUG] hybrid_search_jobs: valid_jobs after filtering = 1140
[DEBUG] hybrid_search_jobs: total results (sorted) = 1140
[DEBUG] find_matches_with_criteria: ranked jobs count = 1140
[DEBUG] find_matches_with_criteria: top_matches count (full list) = 1140
[DEBUG] perform_job_search: matches count = 1140, threshold = 0.7252
[DEBUG] perform_job_search: use_llm_fit = False
[DEBUG] perform_job_search: job_md_list length (jobs to display) = 1140
[DEBUG] perform_job_search: Load More visible = True
[DEBUG] hybrid_search_jobs: valid_jobs after filtering = 57
[DEBUG] hybrid_search_jobs: total results (sorted) = 57
[DEBUG] find_matches_with_criteria: ranked jobs count = 57
[DEBUG] find_matches_with_criteria: top_matches count (full list) = 57
[DEBUG] perform_job_search: matches count = 57, threshold = 0.7401
[DEBUG] perform_job_search: use_llm_fit = True
[DEBUG] perform_job_search: job_md_list length (jobs to display) = 57
[DEBUG] perform_job_search: Load More visible = True
